In [3]:
import kagglehub
import os
import zipfile
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, callbacks
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Extract dataset from the manually downloaded zip file
def extract_dataset():
    # Path to the manually downloaded zip file
    zip_file = 'plant_data.zip'
    extract_folder = "plantvillage_dataset"
    
    # Check if the zip file exists
    if os.path.exists(zip_file) and zip_file.endswith('.zip'):
        # Extract the contents of the zip file
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
        print(f"Dataset extracted to: {extract_folder}")
    else:
        print(f"{zip_file} not found. Please ensure the file is in the same directory as this notebook.")
    return extract_folder

# Call the function to extract the dataset
dataset_dir = extract_dataset()

Dataset extracted to: plantvillage_dataset


In [ ]:
import os
import shutil
import random

# Define the paths
base_dir = os.path.join(dataset_dir, "PlantVillage")
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

# Create train and test directories
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Get all the subdirectories (each representing a class/disease)
class_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]

# Function to split the dataset
def split_dataset():
    for class_folder in class_folders:
        # Get the path to the current class folder
        class_path = os.path.join(base_dir, class_folder)
        
        # Create corresponding class folders in train and test directories
        class_train_dir = os.path.join(train_dir, class_folder)
        class_test_dir = os.path.join(test_dir, class_folder)
        
        # Create directories if they don't exist
        if not os.path.exists(class_train_dir):
            os.makedirs(class_train_dir)
        if not os.path.exists(class_test_dir):
            os.makedirs(class_test_dir)
        
        # Get all image files in the class folder
        image_files = [f for f in os.listdir(class_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        
        # Shuffle the image files for random splitting
        random.shuffle(image_files)
        
        # Split the images (80% for training, 20% for testing)
        split_index = int(0.8 * len(image_files))
        train_files = image_files[:split_index]
        test_files = image_files[split_index:]
        
        # Move the files to the respective directories
        for train_file in train_files:
            shutil.move(os.path.join(class_path, train_file), os.path.join(class_train_dir, train_file))
        
        for test_file in test_files:
            shutil.move(os.path.join(class_path, test_file), os.path.join(class_test_dir, test_file))
        
        print(f"Moved {len(train_files)} images to {class_train_dir} and {len(test_files)} images to {class_test_dir}")

# Run the split function
split_dataset()

print("Dataset split into train and test directories.")

In [ ]:
# Set up ImageDataGenerator for real-time data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Define train and test directories (adjust paths based on your folder structure)
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

# Flow data from directories
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')

In [ ]:
# Function to build and compile a model with a given set of hyperparameters
def build_model(learning_rate, filters_layer1, filters_layer2, dense_neurons, dropout_rate):
    model = models.Sequential()
    
    # First Convolutional Layer
    model.add(layers.Conv2D(filters_layer1, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))

    # Second Convolutional Layer
    model.add(layers.Conv2D(filters_layer2, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Third Convolutional Layer
    model.add(layers.Conv2D(filters_layer2, (3, 3), activation='relu'))

    # Flatten the output
    model.add(layers.Flatten())

    # Fully Connected Layer
    model.add(layers.Dense(dense_neurons, activation='relu'))
    
    # Optional Dropout Layer
    if dropout_rate:
        model.add(layers.Dropout(dropout_rate))
    
    # Output Layer
    model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
from itertools import product

# Define the hyperparameters to test
learning_rates = [0.001, 0.0001]
filters_layer1_list = [32, 64]
filters_layer2_list = [64, 128]
dense_neurons_list = [64, 128]
dropout_rates = [None, 0.3]
batch_sizes = [32, 64]

# Create all combinations of hyperparameters
param_combinations = list(product(learning_rates, filters_layer1_list, filters_layer2_list,
                                   dense_neurons_list, dropout_rates, batch_sizes))

# Store results
results = {}

# Loop through all combinations of parameters
for i, params in enumerate(param_combinations):
    learning_rate, filters_layer1, filters_layer2, dense_neurons, dropout_rate, batch_size = params
    print(f"Testing combination {i + 1}/{len(param_combinations)}: {params}")

    # Build model with current hyperparameters
    model = build_model(learning_rate, filters_layer1, filters_layer2, dense_neurons, dropout_rate)

    # Early stopping callback to avoid overfitting
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=test_generator,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(test_generator, verbose=0)
    
    # Store results
    results[i] = {
        "parameters": {
            "learning_rate": learning_rate,
            "filters_layer1": filters_layer1,
            "filters_layer2": filters_layer2,
            "dense_neurons": dense_neurons,
            "dropout_rate": dropout_rate,
            "batch_size": batch_size
        },
        "test_accuracy": test_acc
    }
    print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# Find the best model based on test accuracy
best_model_idx = max(results, key=lambda x: results[x]["test_accuracy"])
best_model = results[best_model_idx]

print(f"Best Model Parameters: {best_model['parameters']}")
print(f"Best Model Test Accuracy: {best_model['test_accuracy']:.4f}")